##Setup

You will need to make a copy of this notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.

In [1]:
#@title mount your Google Drive
#@markdown Your work will be stored in a folder called `cs285_f2022` by default to prevent Colab instance timeouts from deleting your edits.

import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#@title set up mount symlink

DRIVE_PATH = '/content/gdrive/My\ Drive/cs285_f2022'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/cs285_f2022'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [3]:
#@title apt install requirements

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

!apt update 
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg > /dev/null 2>&1

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 172 kB in 2s (85.7 kB/s)
Reading package lists... Do

In [4]:
#@title clone homework repo
#@markdown Note that this is the same codebase from homework 1,
#@markdown so you may need to move your old `homework_fall2022`
#@markdown folder in order to clone the repo again.

#@markdown **Don't delete your old work though!**
#@markdown You will need it for this assignment.

%cd $SYM_PATH
!git clone https://github.com/berkeleydeeprlcourse/homework_fall2022.git
%cd homework_fall2022/hw4
%pip install -r requirements_colab.txt
%pip install -e .

/content/gdrive/My Drive/cs285_f2022
fatal: destination path 'homework_fall2022' already exists and is not an empty directory.
/content/gdrive/My Drive/cs285_f2022/homework_fall2022/hw4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/My%20Drive/cs285_f2022/homework_fall2022/hw4
  Attempting uninstall: cs285
    Found existing installation: cs285 0.1.0
    Can't uninstall 'cs285'. No files were found to uninstall.
  Running setup.py develop for cs285


In [5]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

# For later
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
)

In [6]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!

import gym
import matplotlib
matplotlib.use('Agg')

env = wrap_env(gym.make("Ant-v4"))

observation = env.reset()
for i in range(10):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;
            
env.close()
print('Loading video...')
show_video()

/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym

Loading video...


## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`cs285_f2022/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

## Run MBPO

In [7]:
#@title imports
import os
import time

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.mbpo_agent import MBPOAgent

%load_ext autoreload
%autoreload 2

In [20]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  def __contains__(self, key):
    return hasattr(self, key)

  env_name = "cheetah-cs285-v0" #@param ["cheetah-cs285-v0", "obstacles-cs285-v0", "reacher-cs285-v0"]
  exp_name = "q6_cheetah_rlen10"#@param
  n_iter = 10 #@param {type:"integer"}

  if env_name == 'reacher-cs285-v0':
    ep_len = 200
  if env_name == 'cheetah-cs285-v0':
    ep_len = 500
  if env_name == 'obstacles-cs285-v0':
    ep_len = 100

  #@markdown batches and steps
  batch_size = 5000 #@param {type: "integer"}
  eval_batch_size = 400 #@param {type: "integer"}
  train_batch_size = 512 #@param {type: "integer"}
  batch_size_initial = 5000 #@param {type: "integer"}

  num_agent_train_steps_per_iter = 1500 #@param {type: "integer"}

  #@markdown MBRL parameters
  ensemble_size = 3 #@param {type:"integer"}
  mpc_horizon = 10 #@param {type:"integer"}
  mpc_num_action_sequences = 1000 #@param {type:"integer"}
  mpc_action_sampling_strategy = 'random' #@param ["random", "cem"]
  cem_iterations = 4 #@param {type: "integer"}
  cem_num_elites = 5 #@param {type: "integer"}
  cem_alpha = 1.0 #@param {type: "raw"}

  #@markdown Learning parameters
  learning_rate = 0.001 #@param {type:"raw"}
  n_layers = 2 #@param {type:"integer"}
  size = 250 #@param {type:"integer"}
  add_sl_noise = True #@param {type:"boolean"}

  #@markdown SAC parameters
  sac_num_agent_train_steps_per_iter = 1 #@param {type: "integer"}
  sac_num_critic_updates_per_agent_update = 1 #@param {type: "integer"}
  sac_num_actor_updates_per_agent_update = 1 #@param {type: "integer"}
  sac_actor_update_frequency = 1 #@param {type: "integer"}
  sac_critic_target_update_frequency = 1 #@param {type: "integer"}
  sac_train_batch_size = 256 #@param {type: "integer"} ##steps used per gradient step
  sac_batch_size = 1500 #@param {type: "integer"} #steps collected per train iteration
  sac_discount = 0.99 #@param {type: "raw"}
  sac_init_temperature = .1 #@param {type: "raw"}
  sac_learning_rate = 0.0003 #@param {type: "raw"}
  sac_n_layers = 2 #@param {type: "integer"}
  sac_size = 256 #@param {type: "integer"}
  sac_n_iter = 1000 #@param {type: "integer"}

  #@markdown MBPO parameters
  mbpo_rollout_length = 10 #@param {type: "integer"}


  #@markdown system
  save_params = False #@param {type: "boolean"}
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

  #@markdown logging
  ## default is to not log video so
  ## that logs are small enough to be
  ## uploaded to gradscope
  video_log_freq = -1 #@param {type: "integer"}
  scalar_log_freq = 1#@param {type: "integer"}


args = Args()

## ensure compatibility with hw1 code
args['train_batch_size'] = args['batch_size']

if args['video_log_freq'] > 0:
  import warnings
  warnings.warn(
      '''\nLogging videos will make eventfiles too'''
      '''\nlarge for the autograder. Set video_log_freq = -1'''
      '''\nfor the runs you intend to submit.''')

In [21]:
#@title create directories for logging

data_path = '''/content/cs285_f2022/''' \
        '''homework_fall2022/hw4/data'''

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = 'hw4_' + args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

print("LOGGING TO: ", logdir)


LOGGING TO:  /content/cs285_f2022/homework_fall2022/hw4/data/hw4_q6_cheetah_rlen10_cheetah-cs285-v0_04-11-2022_23-18-33


In [22]:
#@title Define MBPO Based trainer

class MBPO_Trainer(object):

    def __init__(self, params):

        

        #####################
        ## SET AGENT PARAMS
        #####################

        mb_computation_graph_args = {
            'ensemble_size': params['ensemble_size'],
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
        }
        
        sac_computation_graph_args = {
            'n_layers': params['sac_n_layers'],
            'size': params['sac_size'],
            'learning_rate': params['sac_learning_rate'],
            'init_temperature': params['sac_init_temperature'],
            'actor_update_frequency': params['sac_actor_update_frequency'],
            'critic_target_update_frequency': params['sac_critic_target_update_frequency']
        }
        
        mb_train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
        }

        sac_train_args = {
            'num_agent_train_steps_per_iter': params['sac_num_agent_train_steps_per_iter'],
            'num_critic_updates_per_agent_update': params['sac_num_critic_updates_per_agent_update'],
            'num_actor_updates_per_agent_update': params['sac_num_actor_updates_per_agent_update'],
            'n_iter': params['sac_n_iter'],
            'train_batch_size': params['sac_train_batch_size']
        }

        estimate_advantage_args = {
            'gamma': params['sac_discount'],
        }

        controller_args = {
            'mpc_horizon': params['mpc_horizon'],
            'mpc_num_action_sequences': params['mpc_num_action_sequences'],
            'mpc_action_sampling_strategy': params['mpc_action_sampling_strategy'],
            'cem_iterations': params['cem_iterations'],
            'cem_num_elites': params['cem_num_elites'],
            'cem_alpha': params['cem_alpha'],
        }

        mb_agent_params = {**mb_computation_graph_args, **mb_train_args, **controller_args}
        sac_agent_params = {**sac_computation_graph_args, **estimate_advantage_args, **sac_train_args}
        agent_params = {**mb_agent_params}
        agent_params['sac_params'] = sac_agent_params

        self.params = params
        self.params['agent_class'] = MBPOAgent
        self.params['agent_params'] = agent_params

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            self.params['n_iter'],
            collect_policy = self.rl_trainer.agent.actor,
            eval_policy = self.rl_trainer.agent.actor,
            )

In [23]:
#@title run training

trainer = MBPO_Trainer(args)
trainer.run_training_loop()

########################
logging outputs to  /content/cs285_f2022/homework_fall2022/hw4/data/hw4_q6_cheetah_rlen10_cheetah-cs285-v0_04-11-2022_23-18-33
########################
Using GPU id 0
Using action sampling strategy: random


********** Iteration 0 ************

At timestep:     5010 / 5000
Training agent...

Beginning logging procedure...

Eval_AverageReturn : -1140.80517578125
Eval_StdReturn : 0.0
Eval_MaxReturn : -1140.80517578125
Eval_MinReturn : -1140.80517578125
Eval_AverageEpLen : 501.0
Train_AverageReturn : -2779.1025390625
Train_StdReturn : 422.68048095703125
Train_MaxReturn : -1972.73974609375
Train_MinReturn : -3482.497314453125
Train_AverageEpLen : 501.0
Train_EnvstepsSoFar : 5010
TimeSinceStart : 29.19916319847107
Training Loss : 0.05521625280380249
Initial_DataCollection_AverageReturn : -2779.1025390625
Done logging...




********** Iteration 1 ************

At timestep:     5010 / 5000
Training agent...

Beginning logging procedure...

Eval_AverageReturn : -199.3

In [ ]:
#@markdown You can visualize your runs with tensorboard from within the notebook

## requires tensorflow==2.3.0
%load_ext tensorboard
%tensorboard --logdir /content/cs285_f2022/homework_fall2022/hw4/data/